In [52]:
import pandas as pd

melb_data=pd.read_csv('data/melb_data_ps.csv', sep=',')

In [53]:
melb_df=melb_data.copy()

melb_df.drop(['index','Coordinates'],axis=1,inplace=True)

In [54]:
total_rooms=melb_df['Rooms'] + melb_df['Bathroom'] + melb_df['Bedroom']
display(total_rooms)

0         5
1         5
2         8
3         8
4         8
         ..
13575    10
13576     8
13577     8
13578     9
13579     9
Length: 13580, dtype: int64

In [55]:
melb_df['MeanRoomsSquare']=melb_df['BuildingArea']/total_rooms
display(melb_df['MeanRoomsSquare'])

0        25.200000
1        15.800000
2        18.750000
3        15.750000
4        17.750000
           ...    
13575    12.600000
13576    16.625000
13577    15.750000
13578    17.444444
13579    12.444444
Name: MeanRoomsSquare, Length: 13580, dtype: float64

In [ ]:
diff_area=melb_df['BuildingArea']-melb_df['Landsize']
sum_area=melb_df['BuildingArea']+melb_df['Landsize']

melb_df['AreaRatio']=diff_area/sum_area
display(melb_df['AreaRatio'])

In [ ]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
display(melb_df['Date'])

In [58]:
years_sold = melb_df['Date'].dt.year

print('Min year sold:', years_sold.min())
print('Max year sold:', years_sold.max())
print('Mode year sold:', years_sold.mode()[0])

Min year sold: 2016
Max year sold: 2017
Mode year sold: 2017


In [59]:
#Теперь попробуем понять, на какие месяцы приходится пик продаж объектов недвижимости. 
# Для этого выделим атрибут dt.month и на этот раз занесём результат в столбец MonthSale, 
# а затем найдём относительную частоту продаж для каждого месяца от общего количества продаж — 
# для этого используем метод value_counts() с параметром normalize (вывод в долях)

melb_df['MonthSale'] = melb_df['Date'].dt.month
melb_df['MonthSale'].value_counts(normalize=True)
display(melb_df['MonthSale'].value_counts(normalize=True))


5     0.149411
7     0.145950
9     0.135862
6     0.134757
8     0.114138
11    0.082032
4     0.069882
3     0.049926
12    0.044698
10    0.040574
2     0.032622
1     0.000147
Name: MonthSale, dtype: float64

In [60]:
#Часто бывает такая ситуация, что необходимо вычислять интервалы между двумя временными промежутками. 
# Например, можно вычислить, сколько дней прошло с 1 января 2016 года до момента продажи объекта. 
# Для этого можно просто найти разницу между датами продаж и заявленной датой, представленной в формате datetime

delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01')
#Чтобы превратить количество дней из формата интервала в формат целого числа дней, 
# можно воспользоваться аксессором dt для формата timedelta и извлечь из него атрибут days:

display(delta_days.dt.days)

0        337
1         34
2        428
3        428
4        155
        ... 
13575    603
13576    603
13577    603
13578    603
13579    603
Name: Date, Length: 13580, dtype: int64

In [61]:
#Рассмотрим другой пример. Давайте создадим признак возраста объекта недвижимости в годах на момент продажи. 
# Для этого выделим из столбца с датой продажи год и вычтем из него год постройки здания. 
# Результат оформим в виде столбца AgeBuilding:
melb_df['AgeBuilding']=melb_df['Date'].dt.year-melb_df['YearBuilt']
display(melb_df['AgeBuilding'])
# удалить теперь не нужный столбец

melb_df=melb_df.drop('YearBuilt', axis=1)

0         46
1        116
2        117
3         47
4          2
        ... 
13575     36
13576     22
13577     20
13578     97
13579     97
Name: AgeBuilding, Length: 13580, dtype: int64

In [62]:
#Создайте в таблице melb_df признак WeekdaySale (день недели). 
# Найдите, сколько объектов недвижимости было продано в выходные (суббота и воскресенье), 
# результат занесите в переменную weekend_count. 
# В качестве ответа введите результат вывода переменной weekend_count.
melb_df['WeekdaySale']=melb_df['Date'].dt.dayofweek

weekend_count=melb_df[(melb_df['WeekdaySale']==5)].shape[0]+melb_df[(melb_df['WeekdaySale']==6)].shape[0]


display(melb_df['WeekdaySale'])

0        5
1        3
2        5
3        5
4        5
        ..
13575    5
13576    5
13577    5
13578    5
13579    5
Name: WeekdaySale, Length: 13580, dtype: int64

In [63]:
def get_weekend(weekday):
    if weekday ==5 or weekday==6:
        return 1
   
melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)
display(round(melb_df[melb_df['Weekend']==1]['Price'].mean(), 0))



1081199.0

In [64]:
# На вход данной функции поступает строка с адресом.
def get_street_type(address):
# Создаём список географических пометок exclude_list.
    exclude_list = ['N', 'S', 'W', 'E']
# Метод split() разбивает строку на слова по пробелу.
# В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
# Обрезаем список, оставляя в нём только последний элемент,
# потенциальный подтип улицы, и заносим в переменную street_type.
    street_type = address_list[-1]
# Делаем проверку на то, что полученный подтип является географической пометкой.
# Для этого проверяем его на наличие в списке exclude_list.
    if street_type in exclude_list:
# Если переменная street_type является географической пометкой,
# переопределяем её на второй элемент с конца списка address_list.
        street_type = address_list[-2]
# Возвращаем переменную street_type, в которой хранится подтип улицы.
    return street_type

street_types = melb_df['Address'].apply(get_street_type)
display(street_types)

street_types = street_types.apply(lambda x: 'Av' if x == 'Avenue' else x)
street_types = street_types.apply(lambda x: 'Bvd' if x == 'Boulevard' else x)
street_types = street_types.apply(lambda x: 'Pde' if x == 'Parade' else x)


print(street_types.nunique())

display(street_types.value_counts())

#Для этого к результату метода value_counts применим метод nlargest(),
# который возвращает n наибольших значений из Series. Зададим n=10, 
# т. е. мы хотим отобрать десять наиболее популярных подтипов. 
# Извлечём их названия с помощью атрибута index,
# а результат занесём в переменную popular_stypes:

popular_stypes =street_types.value_counts().nlargest(10).index
print(popular_stypes)

#введём lambda-функцию, которая будет проверять, есть ли строка x в этом перечне, 
# и, если это так, lambda-функция будет возвращать x, 
# в противном случае она будет возвращать строку 'other'. 
# Наконец, применим такую функцию к Series street_types, полученной ранее, 
# а результат определим в новый столбец таблицы StreetType

melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
display(melb_df['StreetType'])
print(melb_df['StreetType'].nunique())


0        St
1        St
2        St
3        La
4        St
         ..
13575    Cr
13576    Dr
13577    St
13578    St
13579    St
Name: Address, Length: 13580, dtype: object

53


St           8012
Rd           2825
Ct            612
Dr            447
Av            361
Gr            311
Pde           226
Pl            169
Cr            152
Cl            100
La             67
Bvd            66
Tce            47
Wy             40
Cct            25
Hwy            24
Sq             11
Crescent        9
Cir             7
Strand          7
Esplanade       6
Grove           5
Mews            4
Grn             4
Fairway         4
Gdns            4
Righi           3
Crossway        3
Esp             2
Victoria        2
Ridge           2
Crofts          2
Grand           1
Summit          1
Hts             1
Athol           1
Highway         1
Outlook         1
Woodland        1
Ave             1
Gra             1
Terrace         1
Eyrie           1
Dell            1
East            1
Loop            1
Nook            1
Glade           1
Qy              1
Cove            1
Res             1
Grange          1
Corso           1
Name: Address, dtype: int64

Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object')


0           St
1           St
2           St
3        other
4           St
         ...  
13575       Cr
13576       Dr
13577       St
13578       St
13579       St
Name: StreetType, Length: 13580, dtype: object

11


In [65]:
popular_selers = melb_df['SellerG'].value_counts().nlargest(49).index
# заменяем значения улиц, не попавших в список популярных на строку 'other'
melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in popular_selers else 'other') 
min_n=melb_df[melb_df['SellerG']=='Nelson']['Price'].min()
min_o=melb_df[melb_df['SellerG']=='other']['Price'].min()
display(min_n/min_o)

1.297709923664122

In [66]:
import pandas as pd
def get_experience(arg):

    arg_list=arg.split()
    month_list=['месяц', 'месяцев', 'месяца']
        
    if arg_list[-1] in month_list:
        if len(arg_list)>4:
            return int(arg_list[-2])+int(arg_list[-4])*12
        else: 
            return int(arg_list[-2])
    else:
        return int(arg_list[-2])

display(int(b[-2])+int(b[-4])*12)

experience_col = pd.Series([
        'Опыт работы 8 лет 3 месяца',
        'Опыт работы 3 года 5 месяцев',
        'Опыт работы 1 год 9 месяцев',
        'Опыт работы 3 месяца',
        'Опыт работы 6 лет'
        ])
display(experience_col.apply(get_experience))

99

0    99
1    41
2    21
3     3
4     6
dtype: int64

In [ ]:



# создаём пустой список
unique_list = []
# пробегаемся по именам столбцов в таблице
for col in melb_df.columns:
    # создаём кортеж (имя столбца, число уникальных значений)
    item = (col, melb_df[col].nunique(),melb_df[col].dtypes) 
    # добавляем кортеж в список
    unique_list.append(item) 
# создаём вспомогательную таблицу и сортируем её
unique_counts = pd.DataFrame(
    unique_list,
    columns=['Column_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique',  ignore_index=True)
# выводим её на экран
display(unique_counts)
display(melb_df.info())

In [75]:
melb_df.drop(['Lattitude', 'Longtitude', 'Landsize'], axis=1, inplace=True)

In [78]:
melb_df.drop(['Address'], axis=1, inplace=True)

In [79]:
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
display(melb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  category      
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  BuildingArea     13580 non-null  float64       
 13  CouncilArea      12211 non-null  category      
 14  Regionname       13580 non-null  categ

None

In [80]:
popular_suburb = melb_df['Suburb'].value_counts().nlargest(119).index

# заменяем значения улиц, не попавших в список популярных на строку 'other'
melb_df['Suburb'] = melb_df['Suburb'].apply(lambda x: x if x in popular_suburb else 'other') 

melb_df['Suburb'].astype('category')
display(melb_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  category      
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  BuildingArea     13580 non-null  float64       
 13  CouncilArea      12211 non-null  category      
 14  Regionname       13580 non-null  categ

None